pip install pymongo

pip install pandas

Connect MongoDB

In [1]:
from pymongo import MongoClient
import json
import datetime
import glob

In [2]:
db_url="mongodb://localhost"
db_host="localhost"
db_port=27017
db_name="mydb"
client = MongoClient(db_url, db_port)
db = client[db_name]

Load paths

In [4]:
path = "..\\RumourTweetDatasets"

In [5]:
topic = "prince-toronto"

In [6]:
rumourPaths = path + "\\" + topic + "\\*"

In [7]:
rumourPaths

'..\\RumourTweetDatasets\\prince-toronto\\*'

In [8]:
tweetsFolder = []
for file in glob.iglob(rumourPaths, recursive=True):
    
    tweetsFolder.append(file)

In [9]:
tweetsFolder

['..\\RumourTweetDatasets\\prince-toronto\\529540733020405760',
 '..\\RumourTweetDatasets\\prince-toronto\\529653029747064832',
 '..\\RumourTweetDatasets\\prince-toronto\\529654186791944192',
 '..\\RumourTweetDatasets\\prince-toronto\\529660296080916480',
 '..\\RumourTweetDatasets\\prince-toronto\\529687410611728384',
 '..\\RumourTweetDatasets\\prince-toronto\\529689679411810304',
 '..\\RumourTweetDatasets\\prince-toronto\\529695367680761856',
 '..\\RumourTweetDatasets\\prince-toronto\\529695483661664257',
 '..\\RumourTweetDatasets\\prince-toronto\\529713467184676864',
 '..\\RumourTweetDatasets\\prince-toronto\\529716453792956416',
 '..\\RumourTweetDatasets\\prince-toronto\\529720273285566464',
 '..\\RumourTweetDatasets\\prince-toronto\\529739968470867968']

Convert structure File 

In [10]:
def DFS(folder, keyOfStructure, structure):
    
    replyingFile = open(folder +  "\\" + keyOfStructure + ".json")
    replyingJson = json.load(replyingFile)
   
    if isinstance(structure,(list)) == True :
        
        replyingJson['reply_by'] = []
        
        return replyingJson
    
        
    rpList = []
    
    for key, value in structure.items():
        
        rpList.append(DFS(folder, key, value))
        
    replyingJson['reply_by'] = rpList
        
    return replyingJson

In [11]:
reactionList = []

for fd in tweetsFolder:
    
    structureFile = open(fd + "\\structure.json")
    structureObj = json.load(structureFile)
    
    sourceFile = open(fd +"\\"+"source-tweets" +"\\" + list(structureObj.keys())[0] + ".json")
    sourceTweetObj = json.load(sourceFile)
    
    reactionsPaths = fd + "\\" + "reactions" 
    
    
    
    if len(list(structureObj.keys())[0]) == 0:
        sourceTweetObj['reply_by'] = []

    else:
        rpList = []
        for key, value in structureObj.get(list(structureObj.keys())[0]).items():
            
            rpList.append(DFS(reactionsPaths, key, value))
            
        sourceTweetObj['reply_by'] = rpList
    
    reactionList.append(sourceTweetObj)

insert reaction list into MongoDB

In [12]:
reationCol = db["reactions"] 
reationCol.drop()
reationCol.insert_many(reactionList)
reationCol.count_documents({})

12